In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=4000)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = None
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
# TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

def getEmailList():
    try:
        results = service.users().messages().list(userId="me").execute()
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        print("Message:")
        for message in messages:
            print(message)

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

Message:
{'id': '18d905ba517a8d6d', 'threadId': '18d905ba517a8d6d'}
{'id': '18d4706edcf6ec88', 'threadId': '18d4706edcf6ec88'}
{'id': '18d016871b43ccaa', 'threadId': '18d016871b43ccaa'}


In [7]:
# Creates a new assistant
assistant = client.beta.assistants.create(
    name = "Gmail Helper",
    instructions = "Your go-to assistant for reading and editing emails. Gmail Helper is a personal assistant dedicated to reading and sending emails, primarily through Google Gmail API integration. Its core functionality includes checking the user’s email to read and summarize emails they receive. It is capable of sending emails to the user's contacts but will always seek confirmation from the user before doing so, ensuring that the user has control over what they send. Gmail Helper maintains a professional demeanor, focusing on providing clear, concise, and accurate scheduling information. It prioritizes user privacy and confidentiality.The assistant is also equipped to guide users through the Google Gmail API’s functionalities and troubleshoot common issues, using language tailored to the user’s level of technical expertise. Overall, Gmail Helper adopts a polite, efficient, and accommodating tone, aiming to make email management and event scheduling as smooth and user-friendly as possible.",
    tools = [{
      "type": "function",
    "function": {
      "name": "getEmailList",
      "description": "Get a list of the user's mailbox contents",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    }
  }],
    model = "gpt-4"
)

In [8]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_I82jGM5MUfKAFH3Fmw8pnS6F', created_at=1707697324, metadata={}, object='thread')


In [9]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What is my latest email?"
)

In [10]:
print(message)

ThreadMessage(id='msg_aaZKDlpyyT9AbZRAimr2pDmS', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is my latest email?'), type='text')], created_at=1707697424, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_I82jGM5MUfKAFH3Fmw8pnS6F')


In [11]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [12]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [13]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [16]:
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)

user: What is my latest email?
assistant: I'm sorry for the confusion, but as a text-based AI model developed by OpenAI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. I don't have the ability to check your Gmail account or any other email platform. My primary function is to provide information and answer questions to the best of my knowledge and abilities. If you have any concerns about privacy or data security, feel free to let me know, and I'll do my best to address them.


In [ ]:
service.close()